In [1]:
from QuarterlyCashFlow import QuarterlyCashFlow

In [2]:
qcf = QuarterlyCashFlow()

In [3]:
qcf.get_latest_quarterly_report("MSFT")

,label,balance,2025-09-30,2024-09-30,cf_category
0,Accounts payable,None,-614.0,-916.0,change_in_accounts_payable
1,Accounts receivable,None,16490.0,14037.0,change_in_accounts_receivable
2,"Acquisition of companies, net of cash acquired...",credit,-578.0,-1849.0,acquisitions_and_intangibles
3,Common stock cash dividends paid,credit,-6169.0,-5574.0,dividends_paid
4,Common stock repurchased,credit,-5650.0,-4107.0,share_repurchases
5,Deferred income taxes,debit,2491.0,-1433.0,deferred_income_taxes
6,"Depreciation, amortization, and other",debit,13061.0,7383.0,depreciation_amortization_and_other
7,Effect of foreign exchange rates on cash and c...,None,-92.0,122.0,effect_of_exchange_rate_on_cash
8,Income taxes,None,-2944.0,1016.0,change_in_income_taxes_payable
9,Inventories,None,-192.0,-373.0,change_in_inventories


In [4]:
from edgar import Company, set_identity

set_identity("Mikolaj Ciuba ciubamikolaj22@gmail.com")

company = Company("MSFT")
    
filing = company.get_filings(form="10-Q").latest()

xbrl = filing.xbrl()
xbrl.statements.cashflow_statement()


                                       Consolidated Statement of Cash Flows                                        
                                                Three Months Ended                                                 
                                  (In millions, except shares and per share data)                                  
                                                                                                                   
                                                                            Sep 30, 2025 (Q3)   Sep 30, 2024 (Q3)  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Operations                                                                                                       
      Net Income                                                                      $27,747             $24,667  
    Adjustments to reconcile net income to net cash from operations:    

In [4]:
stmt = xbrl.statements.cashflow_statement()
df = stmt.to_dataframe()

In [6]:
df["concept"].to_list()

['us-gaap_StatementOfCashFlowsAbstract',
 'us-gaap_CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents',
 'us-gaap_NetCashProvidedByUsedInOperatingActivitiesContinuingOperationsAbstract',
 'us-gaap_NetIncomeLoss',
 'us-gaap_AdjustmentsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesAbstract',
 'us-gaap_DepreciationDepletionAndAmortization',
 'us-gaap_ShareBasedCompensation',
 'us-gaap_OtherNoncashIncomeExpense',
 'us-gaap_IncreaseDecreaseInOperatingCapitalAbstract',
 'us-gaap_IncreaseDecreaseInAccountsReceivable',
 'us-gaap_IncreaseDecreaseInOtherReceivables',
 'us-gaap_IncreaseDecreaseInInventories',
 'us-gaap_IncreaseDecreaseInOtherOperatingAssets',
 'us-gaap_IncreaseDecreaseInAccountsPayable',
 'us-gaap_IncreaseDecreaseInOtherOperatingLiabilities',
 'us-gaap_NetCashProvidedByUsedInOperatingActivities',
 'us-gaap_NetCashProvidedByUsedInInvestingActivitiesContinuingOperationsAbstract',
 'us-gaap_PaymentsToAcquireAvailableForSaleSecuritiesDebt',
 'us-gaap_Pro

In [21]:
import re
import pandas as pd

def tidy_statement_numeric(df):
        
        df = df.copy()
    
        date_cols = [c for c in df.columns if re.match(r"\d{4}-\d{2}-\d{2}", str(c))]
    
        has_value = df[date_cols].apply(pd.to_numeric, errors="coerce").notna().any(axis=1)
        out = df.loc[has_value].copy()
    
        if "preferred_sign" in out.columns:
            for c in date_cols:
                out[c] = pd.to_numeric(out[c], errors="coerce")
            neg = out["preferred_sign"] == -1
            out.loc[neg, date_cols] = -out.loc[neg, date_cols]
    
        keep_cols = ["concept", "label"]
        if "dimension" in out.columns:
            keep_cols.append("dimension")
        if "balance" in out.columns:
            keep_cols.append("balance")
    
        tidy = out[keep_cols + date_cols].sort_values("label").reset_index(drop=True)
        return tidy

In [22]:
tidy = tidy_statement_numeric(df)

In [23]:
tidy

,concept,label,dimension,balance,2025-09-30,2025-06-30
0,us-gaap_AccountsPayableCurrent,Accounts Payable,False,credit,3.258000e+10,2.772400e+10
1,us-gaap_AccountsReceivableNetCurrent,Accounts Receivable,False,debit,5.289400e+10,6.990500e+10
2,us-gaap_EmployeeRelatedLiabilitiesCurrent,Accrued Liabilities,False,None,9.201000e+09,1.370900e+10
3,us-gaap_AccumulatedOtherComprehensiveIncomeLos...,Accumulated Other Comprehensive Income/Loss,False,credit,-2.761000e+09,-3.347000e+09
4,us-gaap_CashAndCashEquivalentsAtCarryingValue,Cash and Cash Equivalents,False,debit,2.884900e+10,3.024200e+10
5,us-gaap_CashCashEquivalentsAndShortTermInvestm...,Cash and Cash Equivalents,False,None,1.020120e+11,9.456500e+10
6,us-gaap_CommonStocksIncludingAdditionalPaidInC...,Common Stock,False,None,1.109640e+11,1.090950e+11
7,us-gaap_DeferredIncomeTaxLiabilitiesNet,Deferred income taxes,False,None,2.852000e+09,2.835000e+09
8,us-gaap_Goodwill,Goodwill,False,debit,1.194970e+11,1.195090e+11
9,us-gaap_FiniteLivedIntangibleAssetsNet,Intangible Assets,False,None,2.123600e+10,2.260400e+10
